In [3]:
%%capture
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U ipywidgets

In [4]:
from huggingface_hub import login
import os

hf_token = os.environ.get("HF_TOKEN")
login(hf_token)

In [ ]:
import sys
import subprocess
import importlib

def check_torch_installation():
    """Check if torch is properly installed"""
    try:
        import torch
        print(f"Torch version: {torch.__version__}")
        print(f"CUDA available: {torch.cuda.is_available()}")
        return True
    except Exception as e:
        print(f"Torch import error: {e}")
        return False

def reinstall_packages():
    """Reinstall torch and transformers in correct order"""
    packages_to_uninstall = ['torch', 'torchvision', 'torchaudio', 'transformers', 'bitsandbytes']
    packages_to_install = [
        'torch --index-url https://download.pytorch.org/whl/cu118',
        'torchvision --index-url https://download.pytorch.org/whl/cu118', 
        'torchaudio --index-url https://download.pytorch.org/whl/cu118',
        'transformers',
        'bitsandbytes'
    ]
    
    print("Uninstalling existing packages...")
    for package in packages_to_uninstall:
        subprocess.run([sys.executable, '-m', 'pip', 'uninstall', package, '-y'])
    
    print("Installing packages in correct order...")
    for package in packages_to_install:
        subprocess.run([sys.executable, '-m', 'pip', 'install'] + package.split())

if __name__ == "__main__":
    if not check_torch_installation():
        print("Attempting to fix installation...")
        reinstall_packages()
        print("Please restart your Python kernel and try importing again.")

Torch import error: partially initialized module 'torch' has no attribute 'autograd' (most likely due to a circular import)
Attempting to fix installation...
Uninstalling existing packages...
Installing packages in correct order...


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_dir = "Qwen/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,   
    device_map="auto",  
    torch_dtype=torch.bfloat16,
    trust_remote_code=True             
)

model.config.use_cache = False
model.config.pretraining_tp = 1

AttributeError: partially initialized module 'torch' has no attribute 'autograd' (most likely due to a circular import)